# Synthetic Label Generation

with `deepseek-r1-distill-qwen-7b`


## Hackernews data

https://huggingface.co/datasets/julien040/hacker-news-posts


In [1]:
import os

import polars as pl
from huggingface_hub import snapshot_download

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"  # turn on HF_TRANSFER

files = snapshot_download(
    repo_id="julien040/hacker-news-posts",
    allow_patterns=["*.parquet"],
    repo_type="dataset",
)

df = pl.scan_parquet(files)
df.head(1).collect()

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

id,title,url,score,time,comments,author
i32,str,str,i32,i32,i32,str
3404047,"""Copyright Office Seeks To Make…","""http://www.techdirt.com/articl…",31,1325175114,2,"""nextparadigms"""


In [2]:
# First, let's check our data
print("Min score:", df.select(pl.col("score")).min().collect().item())
print("Max score:", df.select(pl.col("score")).max().collect().item())

# Then create the distribution
score_distribution = (
    df.select(pl.col("score"))
    .with_columns(
        pl.col("score")
        .cut(
            breaks=[300, 500, 1000, 2000, 5000],
            labels=[
                "0-300",
                "301-500",
                "501-1000",
                "1001-2000",
                "2001-5000",
                "5000+"
            ],
        )
        .alias("score_range")
    )
    .group_by("score_range")
    .len()
    .sort("score_range")
)

score_distribution.collect()

Min score: -1
Max score: 6015


score_range,len
cat,u32
"""0-300""",3977967
"""301-500""",21990
"""501-1000""",9347
"""1001-2000""",1488
"""2001-5000""",163
"""5000+""",2


In [3]:
df = df.filter(
    (pl.col("score") >= 500) & ~pl.col("title").str.contains("Show HN|Ask HN")
).collect()

examples = df.sample(n=10, seed=500).select("title").to_series().to_list()
examples

['Steve Jobs has passed away.',
 "Sellers printing counterfeit books and selling under Amazon's brand",
 'What does the ??!??! operator do in C?',
 'React.js Introduction for People Who Know Just Enough JQuery',
 'Email a Dumpster Fire',
 'A Billionaire Mathematician’s Life of Ferocious Curiosity',
 "NSA shares raw intelligence including Americans' data with Israel",
 'Washington Post Is First Paper to Call for Prosecution of Its Own Source',
 'Uber Fires Anthony Levandowski',
 'AlphaGo Zero: Learning from scratch']

## structured generation


In [4]:
from enum import Enum
from typing import Dict, Union
from pydantic import BaseModel, constr


class HackerNewsLabel(str, Enum):
    DEV = "dev"  # Programming languages, coding practices, software development techniques
    WEB = "web"  # Web development, browsers, frontend/backend frameworks, web standards
    AI_ML = "ai_ml"  # Artificial intelligence, machine learning, data science
    SECURITY = "security"  # Cybersecurity, privacy, vulnerabilities, authentication
    BUSINESS = "business"  # Startups, companies, funding, acquisitions, industry news
    CAREER = "career"  # Job seeking, workplace discussions, professional development
    SCIENCE = "science"  # Research, space exploration, physics, biology, academic papers
    TOOLS = "tools"  # Development tools, utilities, software applications
    CULTURE = "culture"  # Tech industry trends, social impact, community issues
    TECH_NEWS = "tech_news"  # General technology news and updates


class HackerNewsClassification(BaseModel):
    explanation: constr(min_length=40)
    label: HackerNewsLabel

## Prompt


In [5]:
def format_text_as_prompt(title: str) -> str:
    categories = """
    1. dev: Programming languages, coding practices, software development techniques  
    2. web: Web development, browsers, frontend/backend frameworks, web standards  
    3. ai_ml: Artificial intelligence, machine learning, data science  
    4. security: Cybersecurity, privacy, vulnerabilities, authentication  
    5. business: Startups, companies, funding, acquisitions, industry news  
    6. career: Job seeking, workplace discussions, professional development  
    7. science: Research, space exploration, physics, biology, academic papers  
    8. tools: Development tools, utilities, software applications  
    9. culture: Tech industry trends, social impact, community issues  
    10. tech_news: General technology news and updates  
    """

    return f"""Look at the title for the following HackerNews post. Assess what category this post belongs to.

Title: "{title}"

Your role is to classify this HackerNews post title into exactly one category. You should choose out of the following categories:

Categories: {categories}

Return your reasoning and the label you've chosen as a JSON object like this:
```json
{{"explanation": "2-3 clear sentences explaining why this specific category is the best fit",
    "label": "dev" | "web" | "ai_ml" | "security" | "business" | "career" | "science" | "tools" | "culture" | "tech_news"
}}
```
"""

## Starting the server with LM Studio


https://huggingface.co/lmstudio-community/DeepSeek-R1-Distill-Qwen-7B-GGUF


In [6]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
client.models.list()

SyncPage[Model](data=[Model(id='deepseek-r1-distill-qwen-7b@q8_0', created=None, object='model', owned_by='organization_owner'), Model(id='deepseek-r1-distill-qwen-7b@q6_k', created=None, object='model', owned_by='organization_owner'), Model(id='deepseek-r1-distill-llama-8b', created=None, object='model', owned_by='organization_owner'), Model(id='text-embedding-nomic-embed-text-v1.5', created=None, object='model', owned_by='organization_owner')], object='list')

## Generating Labels


In [7]:
messages = [
    {"role": "user", "content": format_text_as_prompt(examples[0])},
]


response = client.beta.chat.completions.parse(
    model="deepseek-r1-distill-qwen-7b",
    messages=messages,
    temperature=0.7,
    response_format=HackerNewsClassification,
)

In [8]:
HackerNewsClassification.model_validate_json(response.choices[0].message.content)

HackerNewsClassification(explanation="... explanation ... | label: ... }<think>Alright, so I've got this task here where I need to classify a HackerNews post based on its title. The title is ", label=<HackerNewsLabel.DEV: 'dev'>)

In [9]:
def predict_label(
    title: str, model: str = "deepseek-r1-distill-qwen-1.5b", client=client
) -> HackerNewsClassification | None:
    try:
        prompt = format_text_as_prompt(title)
        messages = [
            {"role": "user", "content": prompt},
        ]
        response = client.beta.chat.completions.parse(
            model=model,
            messages=messages,
            temperature=0.3,
            response_format=HackerNewsClassification,
        )
        return HackerNewsClassification.model_validate_json(
            response.choices[0].message.content
        )
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [10]:
from rich import print as rich_print

structured_results = []
for example in examples:
    prediction = predict_label(example)
    structured_results.append(prediction)
    rich_print(example)
    rich_print(prediction)
    rich_print("---")

Steve Jobs has passed away.

HackerNewsClassification(
    explanation="... explanation ...”, “label”:“...” }<think>Alright, I'm looking at the title ",
    label=<HackerNewsLabel.DEV: 'dev'>
)

---

Sellers printing counterfeit books and selling under Amazon's brand

HackerNewsClassification(
    explanation="... explanation ...”, “label”: “...” }<think>Alright, let's see. I need to classify the title ",
    label=<HackerNewsLabel.SECURITY: 'security'>
)

...

HackerNewsClassification(
    explanation='..., ... , ... , ... , ... , ... , ... , ... , ... , ... ',
    label=<HackerNewsLabel.DEV: 'dev'>
)

---

AlphaGo Zero: Learning from scratch

HackerNewsClassification(
    explanation="The title 'AlphaGo Zero: Learning from scratch' is focused on a specific AI development project. 
AlphaGo Zero refers to an AI system created by Google that plays the board game Go at a superhuman level. The 
phrase 'Learning from scratch' suggests that this post discusses the process of teaching the AI how to play without
prior knowledge, which involves machine learning and artificial intelligence techniques. Therefore, the category 
should be ai_ml since it pertains directly to advancements in AI and machine learning algorithms.",
    label=<HackerNewsLabel.AI_ML: 'ai_ml'>
)

---

## Room to Think


In [11]:
def predict_label_without_structured(
    title: str, model: str = "deepseek-r1-distill-qwen-1.5b", client=client
) -> str:
    try:
        prompt = format_text_as_prompt(title)
        messages = [
            {"role": "user", "content": prompt},
        ]
        response = client.beta.chat.completions.parse(
            model=model,
            messages=messages,
            temperature=0.7,
        )
        return response.choices[0].message.content

    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [12]:
from rich import print as rich_print

results = []
for example in examples:
    prediction = predict_label_without_structured(example)
    results.append(prediction)
    rich_print(example)
    rich_print(prediction)
    rich_print("---")

Steve Jobs has passed away.

<think>
Alright, I need to figure out which category this Hacker News post belongs to based on the title: "Steve Jobs has 
passed away." The available categories are dev, web, ai_ml, security, business, career, science, tools, culture, 
and tech_news.

First, I'll read through each category to see how they fit. 

- **dev**: This is for programming topics like languages or practices.
- **web**: Related to web development or frameworks.
- **ai_ml**: Artificial intelligence and machine learning.
- **security**: Cybersecurity stuff.
- **business**: Startups, funding, etc.
- **career**: Job-related discussions.
- **science**: Research areas like physics or biology.
- **tools**: Development tools or software applications.
- **culture**: Tech trends or community issues.
- **tech_news**: General tech updates.

The title is about Steve Jobs passing away. Steve Jobs was a significant figure in the tech industry, particularly 
known for Apple. However, the title itself doesn't discuss any technical topic, AI, security details, business 
deals, career moves, scientific research, tools, culture trends, or anything directly related to programming or web
development.

It's more of an obituary or announcement about a notable person in the tech field. Since it's not about any 
specific category like programming or technology news updates per se, but rather announcing someone's death within 
the tech context, I'm leaning towards **tech_news** as the most appropriate category because it covers general tech
updates and industry news.

Other categories don't fit because they're too specific to technical topics. So, "Steve Jobs has passed away" is a 
notable event in the tech world but doesn't fall into any of the other specialized categories listed.
</think>

{"explanation": "The title mentions Steve Jobs passing away, which is an announcement about a notable figure in the
tech industry. While it's not directly technical or news about updates, it falls under general tech news as it 
pertains to a significant event within that field.", "label": "tech_news"}

---

Sellers printing counterfeit books and selling under Amazon's brand

<think>
Okay, so I need to figure out which category the HackerNews post with the title "Sellers printing counterfeit books
and selling under Amazon's brand" belongs to. Let me go through each category one by one.

First, looking at the categories provided: dev, web, ai_ml, security, business, career, science, tools, culture, 
tech_news.

The title is about sellers creating counterfeit books and selling them under Amazon's brand. That sounds more 
related to a business issue because it involves illegal activities in the e-commerce space. Amazon is a big 
company, so if they're involved in counterfeit products, that would be a concern for their business practices or 
possibly an issue with consumer trust.

Let me think about each category:

1. Dev: This is about programming and software development. The title doesn't mention any coding or tech details.
2. Web: It's related to web development or frameworks, but the post isn't discussing any websites or technical 
aspects of web stuff.
3. Ai_ml: That's AI and machine learning; nothing in the title suggests that.
4. Security: While counterfeit books could involve security issues, it's more about legality than cybersecurity 
measures.
5. Business: This seems relevant because selling counterfeit products is a business-related issue, possibly 
affecting Amazon's reputation or compliance with anti-counterfeiting laws.
6. Career: Doesn't fit at all; no job-related content here.
7. Science: Not related to research or academic topics.
8. Tools: No mention of any software tools being used or discussed.
9. Culture: More about trends in the tech industry, which isn't what's happening here.
10. Tech_news: While it does involve technology companies, the issue is more about business practices than general 
tech news.

Putting this together, the issue is a business-related problem because it involves illegal activities affecting a 
company like Amazon and consumer trust. So the best fit would be "business".
</think>

{"explanation": "The post discusses sellers printing counterfeit books under Amazon's brand, which relates to 
illegal activities in the e-commerce sector. This falls under business issues concerning anti-counterfeiting and 
consumer trust, making it most appropriate for the 'business' category.", "label": "business"}

...

<think>
Alright, so I need to figure out which category the Hacker News post with the title "Uber Fires Anthony 
Levandowski" belongs to. Let's break this down step by step.

First, looking at the title: it mentions Uber firing someone named Anthony Levandowski. Uber is a well-known 
company that operates ride-sharing services, so this seems related to a company situation or perhaps an internal 
personnel change.

Now, I'll go through each category one by one to see which fits best:

1. **dev**: This relates to programming, coding, software development practices. The title doesn't mention anything
about technology, coding, or software, so probably not here.
2. **web**: This is about web development, frameworks, browsers, etc. Again, no indication of anything related to 
web stuff in the title.
3. **ai_ml**: Artificial intelligence and machine learning. Not relevant here.
4. **security**: Cybersecurity, privacy issues. While Uber might have security aspects, the title doesn't specify 
any technical security details.
5. **business**: Startups, funding, acquisitions, company news. This fits a bit because Uber is a business entity, 
but the title specifically mentions an employee being fired, which could relate to business operations or HR 
changes within the company.
6. **career**: Job seeking or workplace discussions. Maybe indirectly related since firing someone might affect 
careers, but the title doesn't talk about job opportunities or professional development.
7. **science**: Research areas like physics, biology, etc. Not applicable here.
8. **tools**: Development tools and utilities. Doesn't seem to fit.
9. **culture**: Tech industry trends or community issues. Uber is part of the tech industry, but again, the title 
doesn't mention any cultural aspects.
10. **tech_news**: General technology news. While Uber operates in a tech space, this seems more specific than just
general tech news.

The key here is that the title mentions an action (firing) taken by a company (Uber). Firing someone falls under 
business operations, specifically HR or organizational changes within a company. This is different from, say, a 
technical announcement about a product launch or a security breach. So, it's more about the business aspect of a 
company making personnel changes.

I think "business" is the best fit because it deals with company news and operations, which includes firing an 
employee.
</think>

{"explanation": "The title refers to a personnel change at Uber, indicating it falls under company business 
operations. The category 'business' encompasses such organizational news.", "label": "business"}

---

AlphaGo Zero: Learning from scratch

<think>
Okay, so I need to figure out which category the Hacker News post with the title "AlphaGo Zero: Learning from 
scratch" belongs to. The categories are dev, web, ai_ml, security, business, career, science, tools, culture, and 
tech news.

First, looking at the title itself: it mentions AlphaGo Zero, which I know is an AI that plays Go, a board game. 
"Learning from scratch" suggests it's about how the AI learned without prior knowledge or training data beyond what
was provided during its creation.

AlphaGo Zero is notable because it used self-play and advanced algorithms to teach itself the game effectively. 
This involves machine learning techniques since it's an AI learning through experience, not traditional 
programming.

Now, looking at the categories: dev is about programming and software development, web is web stuff, ai_ml is 
obviously related to AI and ML, security deals with cybersecurity, business is about companies, etc.

Since AlphaGo Zero is a study in AI and machine learning techniques for game playing, it's directly related to AI 
and ML. It doesn't involve web development, security, business, or any other of those areas. So the best fit would 
be ai_ml.
</think>

The title discusses an AI's learning process, which relates to artificial intelligence and machine learning.

```json
{"explanation": "The post discusses AlphaGo Zero, an AI system that learned to play Go without prior training data 
through self-play, highlighting its use of machine learning techniques. This makes the category 'ai_ml' as it 
directly pertains to artificial intelligence and machine learning applications.", "label": "ai_ml"}
```

---

In [13]:
import contextlib
import re
import json

JSON_PATTERN = re.compile(r"```json\n(.*?)```", re.DOTALL)
DIRECT_JSON_PATTERN = re.compile(r"\{[^}]*\}", re.DOTALL)


def extract_json_from_text(text: str) -> tuple[str, dict | None]:
    if match := JSON_PATTERN.search(text):
        json_results = match.group(1)
        with contextlib.suppress(json.JSONDecodeError):
            return text, json.loads(json_results)
    if match := DIRECT_JSON_PATTERN.search(text):
        json_text = match.group(0)
        with contextlib.suppress(json.JSONDecodeError):
            return text, json.loads(json_text)
    return text, None


extract_json_from_text(results[0])

('<think>\nAlright, I need to figure out which category this Hacker News post belongs to based on the title: "Steve Jobs has passed away." The available categories are dev, web, ai_ml, security, business, career, science, tools, culture, and tech_news.\n\nFirst, I\'ll read through each category to see how they fit. \n\n- **dev**: This is for programming topics like languages or practices.\n- **web**: Related to web development or frameworks.\n- **ai_ml**: Artificial intelligence and machine learning.\n- **security**: Cybersecurity stuff.\n- **business**: Startups, funding, etc.\n- **career**: Job-related discussions.\n- **science**: Research areas like physics or biology.\n- **tools**: Development tools or software applications.\n- **culture**: Tech trends or community issues.\n- **tech_news**: General tech updates.\n\nThe title is about Steve Jobs passing away. Steve Jobs was a significant figure in the tech industry, particularly known for Apple. However, the title itself doesn\'t di

## Generate dataset

In [44]:
# Filter DataFrame
filtered_df = df.filter(
    (pl.col("score") >= 300) & ~pl.col("title").str.contains("Show HN|Ask HN") & (pl.col("title").str.len_chars() >= 10)
)

# Get stats on filtered title lengths
filtered_title_lengths = filtered_df["title"].str.len_chars()
filtered_title_length_stats = filtered_title_lengths.describe()
print(f"Filtered title length stats:\n{filtered_title_length_stats}")

# Get filtered score distribution
filtered_score_stats = filtered_df["score"].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
print(f"\nFiltered score distribution:\n{filtered_score_stats}")

# Write filtered DataFrame to Parquet file
filtered_df = filtered_df.sample(n=4000, seed=42)
filtered_df.write_parquet("../data/hackernews_filtered.parquet")

Filtered title length stats:
shape: (9, 2)
┌────────────┬───────────┐
│ statistic  ┆ value     │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 4891.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 46.191576 │
│ std        ┆ 19.263831 │
│ min        ┆ 10.0      │
│ 25%        ┆ 30.0      │
│ 50%        ┆ 46.0      │
│ 75%        ┆ 62.0      │
│ max        ┆ 86.0      │
└────────────┴───────────┘

Filtered score distribution:
shape: (12, 2)
┌────────────┬────────────┐
│ statistic  ┆ value      │
│ ---        ┆ ---        │
│ str        ┆ f64        │
╞════════════╪════════════╡
│ count      ┆ 4891.0     │
│ null_count ┆ 0.0        │
│ mean       ┆ 775.144142 │
│ std        ┆ 351.60203  │
│ min        ┆ 500.0      │
│ …          ┆ …          │
│ 75%        ┆ 842.0      │
│ 90%        ┆ 1170.0     │
│ 95%        ┆ 1423.0     │
│ 99%        ┆ 2228.0     │
│ max        ┆ 5771.0     │
└────────────┴────────────┘
